# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [2]:
# Your code here
cur.execute("""SELECT employeeNumber, lastName, firstName
                    FROM employees
                    JOIN offices
                    USING (officeCode)
                    WHERE city = 'Boston';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName
0,1188,Firrelli,Julie
1,1216,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [3]:
# Your code here
cur.execute("""SELECT officeCode, city, COUNT(employeeNumber) AS num_employees
                    FROM offices 
                    LEFT JOIN employees 
                    USING (officeCode)
                    GROUP BY officeCode
                    HAVING num_employees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [4]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
cur.execute("""SELECT employeeNumber, lastName, firstName, 
                    productName, productDescription, htmlDescription
                    FROM employees e
                    JOIN customers c
                    ON e.employeeNumber = c.salesRepEmployeeNumber 
                    JOIN orders o
                    USING (customerNumber)
                    JOIN orderdetails od
                    USING (orderNumber)
                    JOIN products p
                    USING (productCode)
                    JOIN productlines
                    USING (productLine);""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,productName,productDescription,htmlDescription
0,1165,Jennings,Leslie,1958 Setra Bus,"Model features 30 windows, skylights & glare r...",
1,1165,Jennings,Leslie,1940 Ford Pickup Truck,"This model features soft rubber tires, working...",
2,1165,Jennings,Leslie,1939 Cadillac Limousine,Features completely detailed interior includin...,
3,1165,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger,"This model features, opening doors, detailed e...",
4,1165,Jennings,Leslie,1968 Ford Mustang,"Hood, doors and trunk all open to reveal highl...",
...,...,...,...,...,...,...
2991,1702,Gerard,Martin,1954 Greyhound Scenicruiser,"Model features bi-level seating, 50 windows, s...",
2992,1702,Gerard,Martin,1950's Chicago Surface Lines Streetcar,This streetcar is a joy to see. It has 80 sepa...,
2993,1702,Gerard,Martin,Diamond T620 Semi-Skirted Tanker,This limited edition model is licensed and per...,
2994,1702,Gerard,Martin,1911 Ford Town Car,"Features opening hood, opening doors, opening ...",


In [5]:
# Your code here
# Display each employee's name and sales revenue sorted in descending order
cur.execute("""SELECT employeeNumber, lastName, firstName, SUM(amount) AS total_revenue
                    FROM employees e
                    JOIN customers c
                    ON e.employeeNumber = c.salesRepEmployeeNumber
                    JOIN payments
                    USING (customerNumber)
                    GROUP BY employeeNumber
                    ORDER BY total_revenue DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,total_revenue
0,1370,Hernandez,Gerard,1112003.81
1,1165,Jennings,Leslie,989906.55
2,1401,Castillo,Pamela,750201.87
3,1501,Bott,Larry,686653.25
4,1504,Jones,Barry,637672.65
5,1323,Vanauf,George,584406.80
6,1337,Bondur,Loui,569485.75
7,1611,Fixter,Andy,509385.82
8,1612,Marsh,Peter,497907.16
9,1286,Tseng,Foon Yue,488212.67


In [6]:
# Your code here
# Display each offices and their sales revenues sorted in descending order
cur.execute("""SELECT o.officeCode, o.city, SUM(p.amount) AS total_revenue
                    FROM offices o
                    JOIN employees e
                    USING (officeCode)
                    JOIN customers c
                    ON e.employeeNumber = c.salesRepEmployeeNumber
                    JOIN payments p
                    USING (customerNumber)
                    GROUP BY officeCode
                    ORDER BY total_revenue DESC;""")
df_revenue = pd.DataFrame(cur.fetchall())
df_revenue.columns = [i[0] for i in cur.description]
df_revenue

,officeCode,city,total_revenue
0,4,Paris,2819168.90
1,1,San Francisco,1337439.58
2,7,London,1324325.90
3,3,NYC,1072619.47
4,6,Sydney,1007292.98
5,2,Boston,835882.33
6,5,Tokyo,457110.07


In [7]:
# Your code here
# Display the profit for each order along with associated sales employee's names
# And sort profits in descending order
cur.execute("""SELECT employeeNumber, firstName, lastName, orderNumber,
                    SUM((od.priceEach - p.buyPrice)*od.quantityOrdered) AS profit
                    FROM employees e
                    JOIN customers c
                    ON e.employeeNumber = c.salesRepEmployeeNumber
                    JOIN orders o
                    USING (customerNumber)
                    JOIN orderdetails od
                    USING (orderNumber)
                    JOIN products p
                    USING (productCode)
                    GROUP BY orderNumber
                    ORDER BY profit DESC;""")
df_profit = pd.DataFrame(cur.fetchall())
df_profit.columns = [i[0] for i in cur.description]
df_profit

,employeeNumber,firstName,lastName,orderNumber,profit
0,1621,Mami,Nishi,10165,26465.57
1,1504,Barry,Jones,10310,25106.72
2,1702,Martin,Gerard,10287,24714.51
3,1286,Foon Yue,Tseng,10127,24097.07
4,1286,Foon Yue,Tseng,10204,24003.28
...,...,...,...,...,...
321,1504,Barry,Jones,10158,670.34
322,1612,Peter,Marsh,10409,598.53
323,1621,Mami,Nishi,10387,571.56
324,1401,Pamela,Castillo,10144,467.80


## Level Up: Display the names of every individual product that each employee has sold

In [9]:
# Your code here
cur.execute("""SELECT employeeNumber, lastName, firstName, 
                    productName
                    FROM employees e
                    JOIN customers c
                    ON e.employeeNumber = c.salesRepEmployeeNumber 
                    JOIN orders o
                    USING (customerNumber)
                    JOIN orderdetails od
                    USING (orderNumber)
                    JOIN products p
                    USING (productCode)
                    JOIN productlines
                    USING (productLine);""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,productName
0,1165,Jennings,Leslie,1958 Setra Bus
1,1165,Jennings,Leslie,1940 Ford Pickup Truck
2,1165,Jennings,Leslie,1939 Cadillac Limousine
3,1165,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger
4,1165,Jennings,Leslie,1968 Ford Mustang
...,...,...,...,...
2991,1702,Gerard,Martin,1954 Greyhound Scenicruiser
2992,1702,Gerard,Martin,1950's Chicago Surface Lines Streetcar
2993,1702,Gerard,Martin,Diamond T620 Semi-Skirted Tanker
2994,1702,Gerard,Martin,1911 Ford Town Car


## Level Up: Display the Number of Different Products each employee has sold

In [10]:
# Your code here
cur.execute("""SELECT employeeNumber, lastName, firstName, 
                    COUNT(productName)
                    FROM employees e
                    JOIN customers c
                    ON e.employeeNumber = c.salesRepEmployeeNumber 
                    JOIN orders o
                    USING (customerNumber)
                    JOIN orderdetails od
                    USING (orderNumber)
                    JOIN products p
                    USING (productCode)
                    GROUP BY employeeNumber
                    ORDER BY COUNT(productName) DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,COUNT(productName)
0,1370,Hernandez,Gerard,396
1,1165,Jennings,Leslie,331
2,1401,Castillo,Pamela,272
3,1501,Bott,Larry,236
4,1504,Jones,Barry,220
5,1323,Vanauf,George,211
6,1612,Marsh,Peter,185
7,1611,Fixter,Andy,185
8,1337,Bondur,Loui,177
9,1216,Patterson,Steve,152


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!